# Описание ноутбука
Разномастное тестирование идей, и попытки изобразить что-нибудь дельное

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import pylab
import scipy as sp
from scipy.sparse import csc_matrix, csr_matrix, load_npz, save_npz, issparse
from scipy import absolute
from chord_scripts import plot_spectrogram

In [40]:
def plot_frequencies(datas, title=None, figsize=(15, 7.5), colors=None, titles=None):
    plt.figure(figsize=figsize)
    for idx, data in enumerate(datas):
        if issparse(data):
            data = sp.absolute(data.toarray()[:,0])
        color = colors[idx] if colors else 'b'
        title = titles[idx] if titles else ''
        plt.plot(range(len(data)), data, color, label=title)
    plt.legend()
    plt.xscale('log')
    plt.title = title
    plt.xlabel = "Freqs"
    plt.ylabel = 'Values'
    plt.show()

# Задача: найти метрики для сравнения спектрограмм

Требование к метрике f (для типов аккордов):
* Расстояние между аккордами с одинаковыми типами ~ 0, e.g. $$f(a, b)~=0, при \ a=b  $$
* Расстояние между аккордами разных типов ~ $\inf$ $$f(a,b) ~= \inf, при \ a \neq b$$

## Придумаем метрику на аккордах A:min, A:maj, N

In [4]:
Amins = [load_npz('chords/A_min/{}.npz'.format(idx)).tocsr() for idx in range(126)]

In [5]:
Amajs = [load_npz('chords/A_maj/{}.npz'.format(idx)).tocsr() for idx in range(210)]

In [6]:
Nons =  [load_npz('chords/N/{}.npz'.format(idx)).tocsr() for idx in range(460)]

In [10]:
Amins

[<1025x5129 sparse matrix of type '<type 'numpy.complex128'>'
 	with 44997 stored elements in Compressed Sparse Row format>,
 <1025x4410 sparse matrix of type '<type 'numpy.complex128'>'
 	with 70470 stored elements in Compressed Sparse Row format>,
 <1025x3633 sparse matrix of type '<type 'numpy.complex128'>'
 	with 20408 stored elements in Compressed Sparse Row format>,
 <1025x2067 sparse matrix of type '<type 'numpy.complex128'>'
 	with 25931 stored elements in Compressed Sparse Row format>,
 <1025x2630 sparse matrix of type '<type 'numpy.complex128'>'
 	with 62959 stored elements in Compressed Sparse Row format>,
 <1025x4096 sparse matrix of type '<type 'numpy.complex128'>'
 	with 44666 stored elements in Compressed Sparse Row format>,
 <1025x1289 sparse matrix of type '<type 'numpy.complex128'>'
 	with 14718 stored elements in Compressed Sparse Row format>,
 <1025x2285 sparse matrix of type '<type 'numpy.complex128'>'
 	with 8273 stored elements in Compressed Sparse Row format>,
 

In [24]:
%%time
maj_sums = map(absolute, [sum([chord[:,i] for i in range(chord.shape[1])]) for chord in Amajs[:30]])

CPU times: user 3min 17s, sys: 808 ms, total: 3min 18s
Wall time: 3min 19s


In [63]:
%%time
min_sums = map(absolute, [sum([chord[:,i] for i in range(chord.shape[1])]) for chord in Amins[:30]])

CPU times: user 4min 14s, sys: 224 ms, total: 4min 14s
Wall time: 4min 15s


In [64]:
%%time
non_sums = map(absolute, [sum([chord[:,i] for i in range(chord.shape[1])]) for chord in Nons[:30]])

CPU times: user 2min 3s, sys: 168 ms, total: 2min 3s
Wall time: 2min 4s


Нормализация сумм

In [65]:
maj_sums_norm = np.array(maj_sums)/[chord.shape[1] for chord in Amajs[:30]]
min_sums_norm = np.array(min_sums)/[chord.shape[1] for chord in Amins[:10]]
non_sums_norm = np.array(non_sums)/[chord.shape[1] for chord in Nons[:30]]
#maj_sums_norm = map(lambda x: x/sum(x).toarray()[0][0], maj_sums)
#min_sums_norm = map(lambda x: x/sum(x).toarray()[0][0], min_sums)
#non_sums_norm = map(lambda x: x/sum(x).toarray()[0][0], non_sums)

ValueError: operands could not be broadcast together with shapes (30,) (10,) 

Средние частоты ненормализованные

In [ ]:
maj_avg = csr_matrix(np.sum(sp.sparse.hstack(maj_sums).tocsr().toarray(), axis=1)).T
min_avg = csr_matrix(np.sum(sp.sparse.hstack(min_sums).tocsr().toarray(), axis=1)).T
non_avg = csr_matrix(np.sum(sp.sparse.hstack(non_sums).tocsr().toarray(), axis=1)).T

Средние частоты нормализованные

In [ ]:
maj_avg_norm = csr_matrix(np.sum(sp.sparse.hstack(maj_sums_norm).tocsr().toarray(), axis=1)).T
min_avg_norm = csr_matrix(np.sum(sp.sparse.hstack(min_sums_norm).tocsr().toarray(), axis=1)).T
non_avg_norm = csr_matrix(np.sum(sp.sparse.hstack(non_sums_norm).tocsr().toarray(), axis=1)).T

In [ ]:
type(maj_sums[0].toarray())

In [ ]:
print maj_sums[0].shape
print maj_avg.shape

In [ ]:
plot_frequencies([maj_avg, min_avg, non_avg], figsize=(15,8), colors=['r','b', 'g'], titles=['maj_avg', 'min_avg','non_avg'])
#plot_frequencies([np.log(maj_avg)] )
#plot_frequencies([maj_sums[0]], figsize=(15,5))

In [ ]:
plot_frequencies([maj_avg_norm, min_avg_norm, non_avg_norm], figsize=(15,8), colors=['r','b', 'g'], titles=['maj_avg', 'min_avg','non_avg'])

In [ ]:
plot_frequencies([maj_avg_norm, min_avg_norm], figsize=(15,8), colors=['r','b'], titles=['maj_avg', 'min_avg'])

In [ ]:
plt.figure(figsize=(15, 7.5))
plt.imshow(sp.absolute(Amajs[0].toarray()), origin="lower", cmap="jet",aspect="auto", interpolation="nearest")
#plt.pcolormesh(times, frequencies, spectogram)
#plt.colorbar()
#plt.ylabel('Frequency [Hz]')
#plt.xlabel('Time [sec]')
plt.show()